In [1]:
import pandas as pd
import geopandas as gpd

pd.set_option('display.max_columns', None)

Read the 2022 SAM boundaries as geodf

In [2]:
gdf = gpd.read_file('data/SMALL_AREA_2022_Genralised_20m_view_-4176251332041431098.geojson', driver='GeoJSON')

df = gpd.GeoDataFrame(gdf)

Only keep Dublin SAM boundaries

In [3]:
df = df[df['SA_NUTS3_NAME']=='Dublin'].reset_index(drop=True)

Read our SAM 2022 statistics

In [4]:
stats = pd.read_csv('data/SAPS_2022_Small_Area_270923 (1).csv')

for each boundary, find the corresponding statistic

In [5]:
ids = []
for ii, row in df.iterrows():
    tmp = stats[stats['GUID']==row['SA_GUID_2022']]
    if len(tmp)==1:
        ids.append(tmp.index[0])
    else:
        print('error')
        break

df['SAP_ID']=ids



Bring in the stats associated with this ID

In [6]:
statlist = [
'T1_1AGETT',
'T11_1_FW',
'T11_1_BIW',
'T11_1_BUW',
'T11_1_TDLW',
'T11_1_MW',
'T11_1_CDW',
'T11_1_CPW',
'T11_1_VW',
'T11_1_OTHW',
'T11_1_WMFHW',
'T11_1_NSW',
'T11_1_TW',
'T11_1_FSCCC',
'T11_1_BISCCC',
'T11_1_BUSCCC',
'T11_1_TDLSCCC',
'T11_1_MSCCC',
'T11_1_CDSCCC',
'T11_1_CPSCCC',
'T11_1_VSCCC',
'T11_1_OTHSCCC',
'T11_1_WMFHSCCC',
'T11_1_NSSCCC',
'T11_1_TSCCC',
'T11_1_FT',
'T11_1_BIT',
'T11_1_BUT',
'T11_1_TDLT',
'T11_1_MT',
'T11_1_CDT',
'T11_1_CPT',
'T11_1_VT',
'T11_1_OTHT',
'T11_1_WMFHT',
'T11_1_NST',]

for stat in statlist:
    df[stat]=df['SAP_ID'].apply(lambda x: stats[stat].iloc[x])
    

Does population add up?

In [7]:
df['T1_1AGETT'].sum()

1458154

Yes. Finally rename columns based on glossary gl

In [8]:
gl = pd.read_excel('data/Glossary_Saps_2022_270923.xlsx')

In [10]:
cc = []
for column in df.columns:
    tmp = gl[gl['Column Names']==column]
    if len(tmp)==1:
        cc.append(tmp['Description of Field'].values[0])
    else:
  
        cc.append(column)

df.columns=cc
    

In [11]:
df.to_excel('data/data_to_analyse.xlsx')

Now we get the electoral division data for the dublin electoral areas

In [12]:
eds = pd.read_csv('data/CSO_ELECTORAL_DIVISIONS_2022_Genralised_100m_view_6971988586885511935.csv')

In [13]:
dubs = ['FINGAL','DUN LAOGHAIRE/RATHDOWN','DUBLIN CITY','SOUTH DUBLIN']

In [14]:
eds = eds[eds['COUNTY_ENGLISH'].apply(lambda x: True if x in dubs else False )]

In [15]:
colList = df.columns[28:]

for col in colList:

    pop = []

    for ii, row in eds.iterrows():
        tmp = df[df['ED_GUID']==row['ED_GUID']]
        tmp = tmp[colList]
        pop.append(tmp[col].sum())

    eds[col]=pop

Check the population adds up

In [16]:
eds['Total'].sum()

1458154

In [17]:
gdf2 = gpd.read_file('data/dublin.geojson', driver='GeoJSON')

Problem: WOOD QUAY A is not on the map

In [18]:
eds['MATCH']=eds['ED_ID_STR'].apply(lambda x: str(gdf2[gdf2['OSIED_3441']==x])[0])

In [19]:
eds['MATCH'].value_counts()

     321
E      1
Name: MATCH, dtype: int64

Remove WOOD A and WOOD B. sum the values together. then add back WOOD B into the data

In [20]:
wood_a = eds[eds['ED_ENGLISH']=='WOOD QUAY A']
wood_b = eds[eds['ED_ENGLISH']=='WOOD QUAY B']

In [21]:
for col in colList:
    wood_b[col].iloc[0]+=wood_a[col].iloc[0]

/var/folders/c6/jsjxnj0j6vqbnpqy37npy5dw0000gn/T/ipykernel_84799/13027950.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wood_b[col].iloc[0]+=wood_a[col].iloc[0]


In [22]:
eds = eds[eds['ED_ENGLISH'].apply(lambda x: True if 'WOOD Q' not in x else False)]

In [23]:
eds = pd.concat([eds,wood_b]).reset_index(drop=True)

Make sure population adds up

In [24]:
g=[]
for ii, row in eds.iterrows():
    tmp = gdf2[gdf2['ED_ID']==str(row['ED_ID_STR'])]
    if len(tmp)!=1:
        print('err')
        
        break
    g.append(tmp['geometry'].values[0])

eds['geometry']=g


Turning into a GeoDF

In [25]:
gg = gpd.GeoDataFrame(eds)

In [26]:
gg['ID']=range(len(gg))

Calculate bounding boxes

In [27]:
import numpy as np
import geojson

def get_bounding_box(geometry):
    coords = np.array(list(geojson.utils.coords(geometry)))
    return coords[:,0].min(), coords[:,0].max(),coords[:,1].min(), coords[:,1].max()

In [28]:
bbox = []
for ii, row in gg.iterrows():
    ss = str(gg.envelope.iloc[ii])
    cs = []
    for item in ss.split(','):
        item2 = item.split(' ')
        for item in item2:
            item = item.replace('(','')
            item = item.replace(')','')
            if len(item)>0 and item!='POLYGON':
                cs.append(item)

    cs = list(dict.fromkeys(cs))
    cs.sort()
    item = cs[1]+', '+ cs[2]+', '+ cs[0]+', '+ cs[3]
    bbox.append(item)

gg['bbox']=bbox

Add in columns for work, school and total

In [29]:
work_cols = [col for col in gg.columns if '- Work' in col][:-1]
school_cols = [col for col in gg.columns if '- School' in col][:-1]
total_cols = [col for col in gg.columns if '- Total' in col]

pop = []
for ii, row in gg.iterrows():
    pop.append(row[total_cols].sum())

gg['total_pop']=pop

pop = []
for ii, row in gg.iterrows():
    pop.append(row[work_cols].sum())

gg['work_pop']=pop

pop = []
for ii, row in gg.iterrows():
    pop.append(row[school_cols].sum())

gg['school_pop']=pop

gg['Active travel - Total']=gg['On foot - Total']+gg['Bicycle - Total']
gg['Active travel - Work']=gg['On foot - Work']+gg['Bicycle - Work']
gg['Active travel - School, college or childcare']=gg['On foot - School, college or childcare']+gg['Bicycle - School, college or childcare']


for column in list(work_cols)+['Active travel - Work']:
    gg[column+'_pct']=gg[column]/gg['work_pop']

for column in list(total_cols)+['Active travel - Total']:
    gg[column+'_pct']=gg[column]/gg['total_pop']

for column in list(school_cols)+['Active travel - School, college or childcare']:
    gg[column+'_pct']=gg[column]/gg['school_pop']


In [31]:
gg_2022 = gg.copy()

In [32]:
gg_2022['total_pop'].sum()


1046439

In [33]:
gg_2022['Bicycle - Total'].sum()/gg_2022['total_pop'].sum()


0.06152962571157994

### Now do the same for 2016!

Read the 2022 SAM boundaries as geodf

In [34]:
gdf = gpd.read_file('data/SMALL_AREA_2022_Genralised_20m_view_-4176251332041431098.geojson', driver='GeoJSON')

df = gpd.GeoDataFrame(gdf)

Only keep Dublin SAM boundaries

In [35]:
df = df[df['SA_NUTS3_NAME']=='Dublin'].reset_index(drop=True)

Read our SAM 2016 statistics

In [36]:
stats = pd.read_csv('data/SAPS2016_SA2017.csv')

for each boundary, find the corresponding statistic

In [37]:
ids = []
for ii, row in df.iterrows():
    tmp = stats[stats['GUID']==row['SA_GUID_2016']]
    if len(tmp)==1:
        ids.append(tmp.index[0])
    else:
        print('error')
        break

df['SAP_ID']=ids



Bring in the stats associated with this ID

In [38]:
statlist = [
'T1_1AGETT',
'T11_1_FW',
'T11_1_BIW',
'T11_1_BUW',
'T11_1_TDLW',
'T11_1_MW',
'T11_1_CDW',
'T11_1_CPW',
'T11_1_VW',
'T11_1_OTHW',
'T11_1_WMFHW',
'T11_1_NSW',
'T11_1_TW',
'T11_1_FS',
'T11_1_BIS',
'T11_1_BUS',
'T11_1_TDLS',
'T11_1_MS',
'T11_1_CDS',
'T11_1_CPS',
'T11_1_VS',
'T11_1_OTHS',
'T11_1_WMFHS',
'T11_1_NSS',
'T11_1_TS',
'T11_1_FT',
'T11_1_BIT',
'T11_1_BUT',
'T11_1_TDLT',
'T11_1_MT',
'T11_1_CDT',
'T11_1_CPT',
'T11_1_VT',
'T11_1_OTHT',
'T11_1_WMFHT',
'T11_1_NST',]

for stat in statlist:
    df[stat]=df['SAP_ID'].apply(lambda x: stats[stat].iloc[x])
    

Does population add up?

In [39]:
df['T1_1AGETT'].sum()

1430550

Yes. Finally rename columns based on glossary gl

In [40]:
gl = pd.read_excel('data/Glossary_Saps_2022_270923.xlsx')

In [41]:
cc = []
for column in df.columns:
    tmp = gl[gl['Column Names']==column]
    if len(tmp)==1:
        cc.append(tmp['Description of Field'].values[0])
    else:
        if 'T11_' in column:
            tmp = gl[gl['Column Names']==column+'CCC']
            if len(tmp)==1:
                cc.append(tmp['Description of Field'].values[0])
            else:
                print('err')
                break
        else:
            cc.append(column)

df.columns=cc
    

Now we get the electoral division data for the dublin electoral areas

In [42]:
eds = pd.read_csv('data/CSO_ELECTORAL_DIVISIONS_2022_Genralised_100m_view_6971988586885511935.csv')

In [43]:
dubs = ['FINGAL','DUN LAOGHAIRE/RATHDOWN','DUBLIN CITY','SOUTH DUBLIN']

In [44]:
eds = eds[eds['COUNTY_ENGLISH'].apply(lambda x: True if x in dubs else False )]

In [45]:
colList = df.columns[28:]


In [47]:
for col in colList:

    pop = []

    for ii, row in eds.iterrows():
        tmp = df[df['ED_GUID']==row['ED_GUID']]
        tmp = tmp[colList]
        pop.append(tmp[col].sum())

    eds[col]=pop

Check the population adds up

In [48]:
eds['Total'].sum()

1430550

In [49]:
gdf2 = gpd.read_file('data/dublin.geojson', driver='GeoJSON')


Problem: WOOD QUAY A is not on the map

In [50]:
eds['MATCH']=eds['ED_ID_STR'].apply(lambda x: str(gdf2[gdf2['OSIED_3441']==x])[0])

In [51]:
eds['MATCH'].value_counts()

     321
E      1
Name: MATCH, dtype: int64

Remove WOOD A and WOOD B. sum the values together. then add back WOOD B into the data

In [52]:
wood_a = eds[eds['ED_ENGLISH']=='WOOD QUAY A']
wood_b = eds[eds['ED_ENGLISH']=='WOOD QUAY B']

In [53]:
for col in colList:
    wood_b[col].iloc[0]+=wood_a[col].iloc[0]

/var/folders/c6/jsjxnj0j6vqbnpqy37npy5dw0000gn/T/ipykernel_84799/13027950.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wood_b[col].iloc[0]+=wood_a[col].iloc[0]


In [54]:
eds = eds[eds['ED_ENGLISH'].apply(lambda x: True if 'WOOD Q' not in x else False)]

In [55]:
eds = pd.concat([eds,wood_b]).reset_index(drop=True)

Make sure population adds up

In [56]:
eds['Total'].sum()

1430550

Our data is now correct - let's get the geometry

In [57]:
g=[]
for ii, row in eds.iterrows():
    tmp = gdf2[gdf2['ED_ID']==str(row['ED_ID_STR'])]
    if len(tmp)!=1:
        print('err')
        
        break
    g.append(tmp['geometry'].values[0])

eds['geometry']=g


Turning into a GeoDF

In [58]:
gg = gpd.GeoDataFrame(eds)

In [59]:
gg['ID']=range(len(gg))

Calculate bounding boxes

In [60]:
import numpy as np
import geojson

def get_bounding_box(geometry):
    coords = np.array(list(geojson.utils.coords(geometry)))
    return coords[:,0].min(), coords[:,0].max(),coords[:,1].min(), coords[:,1].max()

In [61]:
bbox = []
for ii, row in gg.iterrows():
    ss = str(gg.envelope.iloc[ii])
    cs = []
    for item in ss.split(','):
        item2 = item.split(' ')
        for item in item2:
            item = item.replace('(','')
            item = item.replace(')','')
            if len(item)>0 and item!='POLYGON':
                cs.append(item)

    cs = list(dict.fromkeys(cs))
    cs.sort()
    item = cs[1]+', '+ cs[2]+', '+ cs[0]+', '+ cs[3]
    bbox.append(item)

gg['bbox']=bbox

In [62]:
work_cols = [col for col in gg.columns if '- Work' in col][:-1]
school_cols = [col for col in gg.columns if '- School' in col][:-1]
total_cols = [col for col in gg.columns if '- Total' in col]

pop = []
for ii, row in gg.iterrows():
    pop.append(row[total_cols].sum())

gg['total_pop']=pop

pop = []
for ii, row in gg.iterrows():
    pop.append(row[work_cols].sum())

gg['work_pop']=pop

pop = []
for ii, row in gg.iterrows():
    pop.append(row[school_cols].sum())

gg['school_pop']=pop

gg['Active travel - Total']=gg['On foot - Total']+gg['Bicycle - Total']
gg['Active travel - Work']=gg['On foot - Work']+gg['Bicycle - Work']
gg['Active travel - School, college or childcare']=gg['On foot - School, college or childcare']+gg['Bicycle - School, college or childcare']


for column in list(work_cols)+['Active travel - Work']:
    gg[column+'_pct']=gg[column]/gg['work_pop']

for column in list(total_cols)+['Active travel - Total']:
    gg[column+'_pct']=gg[column]/gg['total_pop']

for column in list(school_cols)+['Active travel - School, college or childcare']:
    gg[column+'_pct']=gg[column]/gg['school_pop']


now add 2016 data to main df

In [63]:
columns_to_add = work_cols + school_cols + total_cols + list(gg.columns[-39:]) + ['work_pop','total_pop','school_pop']

In [65]:
gg_delta = gg_2022[columns_to_add]-gg[columns_to_add]

In [66]:
tmp = [col for col in gg_delta.columns if 'On foot - Total' in col] 

In [74]:
gg_16 = gg[columns_to_add]

In [75]:
final_columns = [x + '_16' for x in gg_16.columns]

gg_16.columns = final_columns

Finally we add our delta columns which represents the change in each area

In [76]:
newg = pd.concat([gg_2022,gg_16],axis=1)

In [77]:
delta_columns = ['delta_'+x for x in gg_delta.columns]

gg_delta.columns = delta_columns

In [78]:
newg = pd.concat([newg,gg_delta],axis=1)

In [80]:
newg.to_file("data/census_data_total.geojson", driver="GeoJSON")

In [81]:
s= []
for col in newg.columns:
    if '_pct' not in col:
        try:
            s.append(newg[col].sum())
        except:
            s.append('')
    else:
        p = col.replace('_pct','')
        
        yearstring = ''
        if '16' in p:
            yearstring = '_16'

        modestring = 'work_pop'
        if 'School' in p:
            modestring = 'school_pop'
        if 'Total' in p:
             modestring = 'total_pop'

        modestring += yearstring
        test = newg[p].sum()/newg[modestring].sum()

        s.append(test)

  


On foot - Work
Bicycle - Work
Bus, minibus or coach - Work
Train, DART or LUAS - Work
Motorcycle or scooter - Work
Car driver - Work
Car passenger - Work
Van - Work
Other (incl. lorry) - Work
Work mainly at or from home - Work
Not stated - Work
Active travel - Work
On foot - Total
Bicycle - Total
Bus, minibus or coach - Total
Train, DART or LUAS - Total
Motorcycle or scooter - Total
Car driver - Total
Car passenger - Total
Van - Total
Other (incl. lorry) - Total
Work mainly at or from home - Total
Not stated - Total
Active travel - Total
On foot - School, college or childcare
Bicycle - School, college or childcare
Bus, minibus or coach - School, college or childcare
Train, DART or LUAS - School, college or childcare
Motorcycle or scooter - School, college or childcare
Car driver - School, college or childcare
Car passenger - School, college or childcare
Van - School, college or childcare
Other (incl. lorry) - School, college or childcare
Work mainly at or from home - School, college or

In [82]:
sums = pd.DataFrame({'values':s})

In [83]:
sums['col']=newg.columns

In [84]:
sums = sums.T

In [85]:
sums=sums.rename(columns=sums.iloc[1]).drop(sums.index[1]).reset_index(drop=True)


In [86]:
sums = sums[sums.columns[13:]]

sums.drop(['MATCH','geometry','ID','bbox'], axis=1, inplace=True)

In [87]:
tmp = [col for col in sums.columns if 'On foot - Total' in col]

In [90]:
delta_pct = [col for col in sums.columns if 'delta' in col and '_pct' in col]

In [91]:
for col in delta_pct:
    v1 = col.replace('delta_','')
    v2 = v1+'_16'
    v3 = sums[v1].values[0]-sums[v2].values[0]
    sums[col].iloc[0]=v3


In [92]:
sums.to_json('data/sums.json',orient='records')